<a href="https://colab.research.google.com/github/guebin/MP2024/blob/main/quiz/Quiz-3.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

| **항목**               | **허용 여부**        | **비고**                                          |
|------------------------|----------------------|---------------------------------------------------|
| **강의노트 참고**      | 허용                 | 수업 중 제공된 강의노트나 본인이 정리한 자료를 참고 가능       |
| **구글 검색**          | 허용                 | 인터넷을 통한 자료 검색 및 정보 확인 가능        |
| **생성 모형 사용**           | 허용 안함            | 인공지능 기반 도구(GPT 등) 사용 불가            |

In [1]:
import datasets
import transformers
import evaluate
import numpy as np
import pandas as pd 
import torch 
from sklearn.model_selection import train_test_split

/home/cgb3/anaconda3/envs/hf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# `1`. `MBTI` 자료분석 -- 30점

**Kaggle 링크**: [MBTI 데이터셋](https://www.kaggle.com/datasets/datasnaek/mbti-type)

Kaggle에 가입한 후, `archive.zip` 파일을 다운로드하고 Colab에 업로드한 뒤, 아래 명령어를 실행하여 데이터를 불러오라.

```python
!unzip archive.zip
df = pd.read_csv("mbti_1.csv")
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
```

이 데이터는 Myers-Briggs Type Indicator(MBTI) 성격 유형과 관련된 사람들의 텍스트 데이터를 포함하고 있다. 데이터는 사람들이 작성한 게시물의 내용과 MBTI 성격 유형 간의 관계를 분석할 수 있도록 구성되어 있다.

- **총 데이터 수**: 약 8600개의 행
  - **학습용 데이터**: 6940개 행  
  - **테스트용 데이터**: 1735개 행

- **각 행의 구성**:
  - **Type (성격 유형)**: MBTI 성격 유형 (예: INTP, ENFJ 등)
  - **Posts (게시물)**: 해당 사용자가 작성한 게시물

`(1)` 주어진 MBTI 데이터셋을 활용하여 **T(Thinking)** 성향의 사람이 작성한 게시물인지 **F(Feeling)** 성향의 사람이 작성한 게시물인지 구분하는 모델을 학습하라. 

**주의**

1. `df_train`을 훈련자료로, `df_test`를 검증자료로 사용하라.
2. 검증자료에 대한 정확도가 80%이상일 경우만 정답으로 인정한다. 

`(2)` 학습된 모형을 사용하여 아래의 Text에 대한 분류를 수행하라. 

In [16]:
["I prefer making decisions based on logic and facts. Analyzing situations objectively is more important to me than considering emotions.", 
 "When making decisions, I think it’s most important to consider how others feel. Understanding and empathizing with the situation is always my top priority. I feel happiest when relationships are harmonious, and I try to maintain an emotional balance in everything I do."]

['I prefer making decisions based on logic and facts. Analyzing situations objectively is more important to me than considering emotions.',
 'When making decisions, I think it’s most important to consider how others feel. Understanding and empathizing with the situation is always my top priority. I feel happiest when relationships are harmonious, and I try to maintain an emotional balance in everything I do.']

# `2`. `sms_spam` 데이터분석 -- 70점

아래는 Hugging Face의 `sms_spam` 데이터셋을 로드하는 코드이다: 

In [64]:
sms_spam = datasets.load_dataset('sms_spam')['train'].train_test_split(test_size=0.2, seed=42)
sms_spam

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 4459
    })
    test: Dataset({
        features: ['sms', 'label'],
        num_rows: 1115
    })
})

`sms_spam['train'][-3]` 는 훈련 데이터의 마지막에서 세번째 항목을 출력한다. 출력된 샘플은 다음과 같다. 

In [65]:
sms_spam['train'][-3]

{'sms': 'FREE camera phones with linerental from 4.49/month with 750 cross ntwk mins. 1/2 price txt bundle deals also avble. Call 08001950382 or call2optout/J MF\n',
 'label': 1}

출력된 샘플은 딕셔너리 형식으로, sms 항목에는 "**FREE camera phones with linerental from 4.49/month…**"와 같은 문장이 담겨 있다. 이 문장은 스팸(Spam) 메시지로 분류되며, `label` 항목에 1로 저장되어 있다.

label이 나타내는 분류는 다음과 같이 정의된다:

In [66]:
sms_spam['train'].features['label'].names

['ham', 'spam']

분류 레이블은 총 2가지로 나누며, 각각의 레이블은 다음과 같이 정의된다:

In [68]:
{0: 'ham', 1: 'spam'}

{0: 'ham', 1: 'spam'}

따라서, 문장 "**FREE camera phones with linerental…**"의 분류는 `label`이 1이므로, 스팸(Spam)에 해당한다.

`(1)` 아래의 코드를 참고하여 `sms_spam` 자료를 전처리하라. -- 10점

```Python
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased") 
def preprocess_function(examples):
    return tokenizer(examples["sms"], truncation=True)
```

전처리된 결과의 샘플은 아래와 같다. 

In [169]:
sms_spam_preprocessed['train'][9]

{'sms': 'Headin towards busetop\n',
 'label': 0,
 'input_ids': [101, 2132, 2378, 2875, 3902, 18903, 2361, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

`(2)` 적절한 모델을 설계하고 `sms_spam` 데이터를 분석하여 스팸 여부를 판별하는 코드를 작성하라. -- 30점

1. `sms_spam['train']`을 훈련자료로, `sms_spam['test']`를 검증자료로 사용하라.
2. 검증자료에 대한 정확도가 90%이상일 경우만 정답으로 인정한다. 

`(3)` 아래의 자료에 대한 loss를 계산하라. -- 30점 

In [191]:
# sms_spam_preprocessed['test'][::100]

**Note1:** 2-(2)를 풀지 않은상태에서 계산해도 정답으로 인정  

**Note2:** `loss`가 출력되는 어떠한 형태의 답안도 정답으로 인정, 예를들면 아래와 같은 결과를 얻을 경우도 정답으로 인정. 

```
PredictionOutput(predictions=array([[-2.3364387,  2.866232 ],
       [ 3.2778776, -2.9869947],
       [ 3.0951078, -2.8594062],
       [ 2.7923744, -2.6019576],
       [ 3.460438 , -3.1327899],
       [ 3.395659 , -3.091136 ],
       [ 3.403764 , -3.1232061],
       [ 3.2923138, -2.9569058],
       [ 3.1802518, -2.8368335],
       [ 3.3302088, -2.9617522],
       [ 3.394341 , -3.0325623],
       [ 3.1710148, -2.9316459]], dtype=float32),
label_ids=array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
metrics={'test_loss': 0.0024105568882077932, 'test_accuracy': 1.0, 'test_runtime': 1.1632, 'test_samples_per_second': 10.317, 'test_steps_per_second': 0.86})
```